# 🔌 Plugins (with semantic functions)

### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"

#!import Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha

### 🦜 Let's building a plugin consisting of semantic functions.

### Now the kernel contains all plugins ever added, unless we start removing them.
Entire plugins or individual functions can be removed from the kernel anytime.

Semantic code gets executed in the LLM.
We can ground the AI in knowledge of the world outside LLM to know about today's date just by giving it a plugin that can access native code capabilities.

### 📋 Let's prepare the execution settings and the kernel arguments.

In [2]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 2000,
     ResultsPerPrompt = 1,
     Temperature = 0.1
};

var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "Christmas"
};

### 🪴 This simple bit of "grounding" reduces so-called 'hallucination' with LLMs

In this first example, we invoke the kernel with a prompt that asks the AI for information it cannot provide. It has no idea what today might be for you, so it's likely to hallucinate.

In [3]:
var promptTemplate = "How many days until {{$input}}?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

Since my knowledge was last updated in March 2023 and today's date is not provided, I cannot give you the exact number of days until Christmas. However, Christmas is celebrated on December 25th every year. To find out how many days are left until Christmas, you can simply subtract today's date from December 25th of the current year.

For example, if today were November 1st, you would have:

December 25th - November 1st = 54 days until Christmas

Please check today's date and perform a similar calculation to find out the number of days remaining until Christmas.

In this second example, the plugin we've added `DateTimePlugin` is used in the templated prompt. The native function GetCurrentDate is invoked to enrich the prompt context with current date.

In [4]:
var promptTemplate = "Considering that today date is {{date.GetCurrentDate}}, how many days until {{$input}}?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

Error: System.Collections.Generic.KeyNotFoundException: The plugin collection does not contain a plugin and/or function with the specified names. Plugin name - 'date', function name - 'GetCurrentDate'.
   at Microsoft.SemanticKernel.KernelPluginExtensions.GetFunction(IReadOnlyKernelPluginCollection plugins, String pluginName, String functionName)
   at Microsoft.SemanticKernel.Kernel.InvokeAsync(String pluginName, String functionName, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.TemplateEngine.CodeBlock.RenderFunctionCallAsync(FunctionIdBlock fBlock, Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.KernelPromptTemplate.RenderAsync(List`1 blocks, Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.KernelFunctionFromPrompt.RenderPromptAsync(Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken

### 👀 Let's check the rendered prompt.

In [5]:
#!import Helpers/PrintHelper.cs

await PrintHelper.PrintRenderedPromptAsync(kernel, promptTemplate, kernelArguments);

Error: System.Collections.Generic.KeyNotFoundException: The plugin collection does not contain a plugin and/or function with the specified names. Plugin name - 'date', function name - 'GetCurrentDate'.
   at Microsoft.SemanticKernel.KernelPluginExtensions.GetFunction(IReadOnlyKernelPluginCollection plugins, String pluginName, String functionName)
   at Microsoft.SemanticKernel.Kernel.InvokeAsync(String pluginName, String functionName, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.TemplateEngine.CodeBlock.RenderFunctionCallAsync(FunctionIdBlock fBlock, Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.KernelPromptTemplate.RenderAsync(List`1 blocks, Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken)
   at Submission#7.PrintHelper.PrintRenderedPromptAsync(Kernel kernel, String promptTemplate, KernelArguments kernelArguments)
   at Submission#8.<<In

### 🚰 For a better user experience we can may want to use with streaming 🚰.

In [6]:
await foreach (var token in kernel.InvokePromptStreamingAsync(promptTemplate, kernelArguments))
{
    Console.Write(token);
}

Error: System.Collections.Generic.KeyNotFoundException: The plugin collection does not contain a plugin and/or function with the specified names. Plugin name - 'date', function name - 'GetCurrentDate'.
   at Microsoft.SemanticKernel.KernelPluginExtensions.GetFunction(IReadOnlyKernelPluginCollection plugins, String pluginName, String functionName)
   at Microsoft.SemanticKernel.Kernel.InvokeAsync(String pluginName, String functionName, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.TemplateEngine.CodeBlock.RenderFunctionCallAsync(FunctionIdBlock fBlock, Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.KernelPromptTemplate.RenderAsync(List`1 blocks, Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.KernelFunctionFromPrompt.RenderPromptAsync(Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken

### There are many ways to build plugins in SK

### You can build them and then add them to the kernel

In [7]:
// kernel.CreatePluginFromFunctions()

// kernel.CreatePluginFromObject()

// kernel.CreatePluginFromPromptDirectory()

// kernel.CreatePluginFromType<>()


// kernel.Plugins.Add(plugin);

### Or you can add them directly (import)...

In [8]:
// kernel.ImportPluginFromPromptDirectory();           // semantic

// kernel.ImportPluginFromFunctions()                  // native

// kernel.ImportPluginFromPromptDirectory()            // semantic

// kernel.ImportPluginFromObject()                     // native

// kernel.ImportPluginFromType<>()                     // native or semantic from native

// kernel.ImportPluginFromOpenAIAsync                  // semantic from OpenAI plugin